In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout,BatchNormalization
from keras.layers import LSTM,GRU
from keras.utils import np_utils
import nltk
import string

In [3]:
text= nltk.corpus.gutenberg.raw('blake-poems.txt')

text=text.lower()
text=text.replace("blake","")
text=text.replace("william","")
text=text.replace("[","")
text=text.replace("]","")
text=text.replace(".","")
text=text.replace(",","")
text=text.replace("!","")
text=text.replace("?","")
def clean_text(text):
  text = "".join(v for v in text if v not in string.punctuation).lower()
  text = text.encode("utf8").decode("ascii",'ignore')
  return text
text=[clean_text(x) for x in text]

In [4]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [5]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
     sequence = text[i:i + seq_length]
     label =text[i + seq_length]
     X.append([char_to_n[char] for char in sequence])
     Y.append(char_to_n[label])

In [6]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [7]:
model = Sequential()

model.add(Dense(400, input_shape=(X_modified.shape[1], X_modified.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [8]:
model.fit(X_modified,Y_modified,epochs=60,batch_size=50)

In [13]:
# pick a random seed
import sys

pattern=[char_to_n[value] for value in (str(input("Enter few words:")))]
print ("Seed:")
print ("\"", ''.join([n_to_char[value] for value in pattern]), "\"")
# generate characters

for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(characters))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = n_to_char[index]
    seq_in = [n_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]